In [1]:
import numpy as np  
import pandas as pd  
import re
from nltk.stem import PorterStemmer,WordNetLemmatizer


In [2]:
df = pd.read_csv('./dataset/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
df.shape
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [4]:
df.dtypes

id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [5]:
df.describe()


,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [7]:
# finding all the rows where the sum of labels is zero i.e the comment is a Clean comment
rowsums=df.iloc[:,2:].sum(axis=1)
df['clean']=(rowsums==0)
df['clean'].sum()

143346

In [8]:
# Total no.of toxic comments
len(df[df['toxic']==1])

15294

In [9]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,True
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,True
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,True
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,True
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,True
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,True
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,False
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,True
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,True
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,True


In [10]:
comment = df['comment_text']

In [11]:
# creating a numpy array of the length of each comment in the dataset.
x = np.array([len(comment[i]) for i in range(comment.shape[0])])

In [12]:
label = df[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']]
print(label.head(10))
label = label.values

   toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0              0
1      0             0        0       0       0              0
2      0             0        0       0       0              0
3      0             0        0       0       0              0
4      0             0        0       0       0              0
5      0             0        0       0       0              0
6      1             1        1       0       1              0
7      0             0        0       0       0              0
8      0             0        0       0       0              0
9      0             0        0       0       0              0


In [13]:
label.shape

(159571, 6)

In [14]:
trim_comments = [comment[i] for i in range(comment.shape[0]) if len(comment[i])<=400 ]

# creating corresponding labels for those comments
my_labels = np.array([label[i] for i in range(comment.shape[0]) if len(comment[i])<=400 ])

In [15]:
my_labels[:10, :]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=int64)

In [16]:
import string
print(string.punctuation)
punctuation_edit = string.punctuation.replace('\'','') +"0123456789"
print (punctuation_edit)
outtab = "                                         "
trantab = str.maketrans(punctuation_edit, outtab)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [17]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Adding alphabets to the set
for i in range(ord('a'),ord('z')+1):
    stop_words.add(chr(i))
print(stop_words)

{'until', 'out', 'has', 'any', 'such', 'these', 'its', 'k', 'p', "didn't", 'against', 'other', 'he', 'not', 'g', 'couldn', 'both', 'her', 'at', 'just', "don't", 'd', "hadn't", 'y', 'j', 'e', 'will', 'myself', 'while', 'all', 'do', 'under', 'aren', 't', 'wouldn', "shan't", 'mightn', 'about', 'again', 'during', 'over', 'only', 'now', "couldn't", 're', 'b', 'r', 'haven', 'of', "wasn't", 'then', 'yours', 'f', 'l', 'u', 'being', 'z', 'to', 'because', "you're", 'this', 'our', 'but', 'be', 'the', 'if', 'doesn', 'below', 'shouldn', 'q', 'than', 'themselves', 'why', "needn't", 'too', 'into', 'how', 'here', 'an', 'w', "she's", "shouldn't", 'in', 'each', 'further', 'needn', 'we', 'what', 'whom', 'your', 'ours', "you'd", 'theirs', 'am', 'between', 'and', 'been', 'doing', 'does', 'on', 'by', 'you', 'from', 'down', 'their', 'm', 'after', "hasn't", 'it', "you'll", 'once', 'yourselves', "it's", 'very', 'those', 'no', 'few', 'through', 'him', "weren't", 'own', 'up', 'ain', 'mustn', 'are', "doesn't", 't

In [18]:
# Stemming and Lemmatizing
from nltk.stem import WordNetLemmatizer, PorterStemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [19]:
# Looping through all the comments and processing them through the functions defined above.
# import nltk
# nltk.download('wordnet')
for i in range(len(trim_comments)):
    trim_comments[i] = trim_comments[i].lower().translate(trantab)
    word_list = []
    for word in trim_comments[i].split():
        if not word in stop_words:
            word_list.append(stemmer.stem(lemmatizer.lemmatize(word,pos="v")))
    trim_comments[i]  = " ".join(word_list)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
 # Convert set to list
stop_words_list = list(stop_words)
#create object supplying our custom stop words
count_vector = CountVectorizer(stop_words=stop_words_list)
#fitting it to converts comments into bag of words format
tf = count_vector.fit_transform(trim_comments[:20000]).toarray()

In [22]:
tf.shape

(20000, 24688)

In [23]:
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion)
    X_train = matrix[ratio:,:]
    X_test =  matrix[:ratio,:]
    Y_train = target[ratio:,:]
    Y_test =  target[:ratio,:]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = shuffle(tf, my_labels[:20000],3)

print(X_test.shape)
print(X_train.shape)
print(Y_train.shape)

(6666, 24688)
(13334, 24688)
(13334, 6)


In [24]:
class SVM:
    def __init__(self, C=1.0, tol=0.01, max_passes=5):
        self.C = C
        self.tol = tol
        self.max_passes = max_passes
        self.models = []

    def fit(self, X, y):
        num_labels = y.shape[1]
        for i in range(num_labels):
            print(f"Training SVM for label {i + 1}")
            svm_label = self.train_one_label(X, y[:, i])
            self.models.append(svm_label)
        return self

    def train_one_label(self, X, y):
        m, n = X.shape
        alphas = np.zeros(m)
        b = 0

        passes = 0
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(m):
                Ei = self.decision_function(X[i], X, alphas, b, y) - y[i]
                if (y[i] * Ei < -self.tol and alphas[i] < self.C) or \
                        (y[i] * Ei > self.tol and alphas[i] > 0):
                    j = self.random_index(i, m)
                    Ej = self.decision_function(X[j], X, alphas, b, y) - y[j]

                    alpha_i_old, alpha_j_old = alphas[i], alphas[j]

                    if y[i] != y[j]:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(self.C, self.C + alphas[j] - alphas[i])
                    else:
                        L = max(0, alphas[i] + alphas[j] - self.C)
                        H = min(self.C, alphas[i] + alphas[j])

                    if L == H:
                        continue

                    eta = 2 * X[i] @ X[j] - X[i] @ X[i] - X[j] @ X[j]
                    if eta >= 0:
                        continue

                    alphas[j] -= y[j] * (Ei - Ej) / eta
                    alphas[j] = max(L, min(alphas[j], H))

                    if abs(alphas[j] - alpha_j_old) < 1e-5:
                        continue

                    alphas[i] += y[i] * y[j] * (alpha_j_old - alphas[j])
                    b1 = b - Ei - y[i] * (alphas[i] - alpha_i_old) @ X[i] @ X[j].T - \
                         y[j] * (alphas[j] - alpha_j_old) @ X[j] @ X[i].T
                    b2 = b - Ej - y[i] * (alphas[i] - alpha_i_old) @ X[i] @ X[j].T - \
                         y[j] * (alphas[j] - alpha_j_old) @ X[j] @ X[j].T
                    if 0 < alphas[i] < self.C:
                        b = b1
                    elif 0 < alphas[j] < self.C:
                        b = b2
                    else:
                        b = (b1 + b2) / 2

                    num_changed_alphas += 1

            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

        return {'alphas': alphas, 'b': b}

    def decision_function(self, x, X, alphas, b, y):
        return np.sum(alphas * y * (X @ x.T)) + b

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))
        for i, model in enumerate(self.models):
            alphas = model['alphas']
            b = model['b']
            predictions[:, i] = np.sign(np.sum(alphas * y * (X @ X.T), axis=1) + b)
        return predictions

    def random_index(self, i, m):
        j = i
        while j == i:
            j = np.random.randint(0, m)
        return j



In [25]:
# Convert labels to -1 and 1
# def convert_labels(labels):
#     return 2 * labels - 1
def convert_labels(labels):
    return 2 * labels - 1

In [26]:

# Train the SVM
svm = SVM()
svm.fit(X_train, Y_train)


Training SVM for label 1


KeyboardInterrupt: 

In [31]:
 # Assuming SVM_module is the module where SVM class is defined

# Instantiate SVM
svm = SVM()

# List of unique labels in your dataset
unique_labels = [1, 2, 3, 4, 5, 6]

# Training loop
for label in unique_labels:
    # Filter the training data for the current label
    mask = (Y_train == label)
    X_label = X_train[mask]
    Y_label = Y_train[mask]

    # Ensure Y_label is in a one-dimensional format for binary classification
    Y_label = Y_label[:, 0] if Y_label.ndim > 1 else Y_label

    # Train SVM for the current label
    print(f"Training SVM for label {label}")
    svm.fit(X_label, Y_label)

# After training is complete, you can use the trained model as needed


IndexError: boolean index did not match indexed array along dimension 1; dimension is 24688 but corresponding boolean dimension is 6